# Labeling BLE Data with Room Information

## Objective
Match each BLE sensor reading with its corresponding room label based on timestamp alignment.

## Approach
We perform a **timestamp-based join** between the BLE sensor data and location labels:

1. **Merge Strategy**: Use `pd.merge_asof()` with `direction='backward'` to find the most recent label where `started_at <= BLE_timestamp`
2. **Validation**: Ensure `BLE_timestamp <= finished_at` to guarantee the timestamp falls within the labeled time range
3. **Filtering**: Keep only records with valid room labels (drop unlabeled data)

## Why Drop Unlabeled Records?

**Decision: Drop 34% of BLE data without matching labels**

### Rationale:
- **Supervised Learning Requirement**: We need labeled data to train a room classification model
- **Data Collection Design**: 
  - User 90 (caregiver) continuously collected BLE sensor data
  - User 97 (labeler) selectively annotated specific location visits
  - This creates gaps where sensor data exists but no ground truth label is available
- **Unlabeled records represent**:
  - Transition periods between rooms
  - Times when the labeler wasn't actively tracking
  - Areas outside the scope of this challenge
- **Not a preprocessing error**: The 34% gap is inherent to the dataset design, not a mistake in our code

### Result:
- Original BLE records: ~1.67M
- Labeled records retained: ~1.10M (66%)
- Records dropped: ~0.57M (34%)

## Data Cleaning Steps:
1. ✅ Convert timestamps to datetime format
2. ✅ Match BLE readings with room labels using time ranges
3. ✅ Validate timestamp falls within [started_at, finished_at]
4. ✅ Drop records without valid labels
5. ✅ Remove unnecessary columns (power, started_at, finished_at)

In [1]:
import pandas as pd
import numpy as np

# Load the datasets
label_df = pd.read_csv("../cleaned_dataset/cleaned_label_loc.csv")
ble_df = pd.read_csv("../cleaned_dataset/cleaned_ble_data.csv")

print("=" * 60)
print("STEP 1: Loading and Converting Timestamps")
print("=" * 60)

# Convert timestamp columns to datetime
label_df['started_at'] = pd.to_datetime(label_df['started_at'])
label_df['finished_at'] = pd.to_datetime(label_df['finished_at'])
ble_df['timestamp'] = pd.to_datetime(ble_df['timestamp'])

print(f"✓ Loaded {len(label_df)} label records")
print(f"✓ Loaded {len(ble_df)} BLE records")
print(f"✓ Label time range: {label_df['started_at'].min()} to {label_df['finished_at'].max()}")
print(f"✓ BLE time range: {ble_df['timestamp'].min()} to {ble_df['timestamp'].max()}")

print("\n" + "=" * 60)
print("STEP 2: Matching BLE Records with Room Labels")
print("=" * 60)

# Sort both dataframes for efficient merging
label_df = label_df.sort_values('started_at').reset_index(drop=True)
ble_df = ble_df.sort_values('timestamp').reset_index(drop=True)

# Merge BLE data with labels using timestamp matching
labelled_ble_df = pd.merge_asof(
    ble_df,
    label_df[['started_at', 'finished_at', 'room']],
    left_on='timestamp',
    right_on='started_at',
    direction='backward'
)

print(f"✓ Performed merge_asof join")

print("\n" + "=" * 60)
print("STEP 3: Validating Timestamp Ranges")
print("=" * 60)

# CRITICAL: Ensure timestamp is within [started_at, finished_at]
# Only keep records where: started_at <= timestamp <= finished_at
before_validation = len(labelled_ble_df)
labelled_ble_df = labelled_ble_df[
    labelled_ble_df['timestamp'] <= labelled_ble_df['finished_at']
].copy()
after_validation = len(labelled_ble_df)

print(f"✓ Before validation: {before_validation} records")
print(f"✓ After validation: {after_validation} records")
print(f"✓ Dropped {before_validation - after_validation} records outside labeled time ranges")
print(f"✓ Retention rate: {after_validation/len(ble_df)*100:.1f}%")

print("\n" + "=" * 60)
print("STEP 4: Cleaning Up Columns")
print("=" * 60)

# Drop unnecessary columns
columns_before = labelled_ble_df.columns.tolist()
labelled_ble_df = labelled_ble_df.drop(columns=['started_at', 'finished_at', 'power'])
columns_after = labelled_ble_df.columns.tolist()

print(f"✓ Dropped columns: started_at, finished_at, power")
print(f"✓ Remaining columns: {columns_after}")

print("\n" + "=" * 60)
print("FINAL DATASET SUMMARY")
print("=" * 60)

print(f"\nTotal labeled BLE records: {len(labelled_ble_df):,}")
print(f"Percentage of original data retained: {len(labelled_ble_df)/len(ble_df)*100:.1f}%")
print(f"Number of unique beacons: {labelled_ble_df['mac address'].nunique()}")
print(f"Number of unique rooms: {labelled_ble_df['room'].nunique()}")

print(f"\n{'Room Distribution:'}")
print("-" * 60)
room_counts = labelled_ble_df['room'].value_counts().sort_index()
for room, count in room_counts.items():
    print(f"  {room:.<30} {count:>10,} records ({count/len(labelled_ble_df)*100:>5.1f}%)")

print("\n" + "=" * 60)
print("✓ Labeling Complete - Ready for Feature Engineering")
print("=" * 60)

# Display first few records
print("\nSample of labeled data:")
print(labelled_ble_df.head(5))

STEP 1: Loading and Converting Timestamps
✓ Loaded 451 label records
✓ Loaded 1673395 BLE records
✓ Label time range: 2023-04-10 14:21:46+09:00 to 2023-04-13 17:18:25+09:00
✓ BLE time range: 2023-04-10 13:00:00+09:00 to 2023-04-13 13:56:21+09:00

STEP 2: Matching BLE Records with Room Labels
✓ Performed merge_asof join

STEP 3: Validating Timestamp Ranges
✓ Before validation: 1673395 records
✓ After validation: 1099957 records
✓ Dropped 573438 records outside labeled time ranges
✓ Retention rate: 65.7%

STEP 4: Cleaning Up Columns
✓ Dropped columns: started_at, finished_at, power
✓ Remaining columns: ['timestamp', 'mac address', 'RSSI', 'room']

FINAL DATASET SUMMARY

Total labeled BLE records: 1,099,957
Percentage of original data retained: 65.7%
Number of unique beacons: 23
Number of unique rooms: 22

Room Distribution:
------------------------------------------------------------
  501...........................      4,043 records (  0.4%)
  502...........................      2,126 

In [3]:
# save the labelled ble data df to a new csv file for future use
labelled_ble_df.to_csv("../cleaned_dataset/labelled_ble_data.csv", index=False)